In [1034]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import unidecode
import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt 

In [1035]:
team_dict = {'Boston Red Sox' : 'BOS', 'Chicago White Sox' : 'CHW', 'Cincinnati Reds' : 'CIN', 'Houston Astros' : 'HOU',\
            'Kansas City Royals': 'KCR', 'Los Angeles Angels' : 'LAA', 'Miami Marlins' : 'MIA', 'Milwaukee Brewers' : 'MIL',\
            'New York Mets' : 'NYM', 'San Francisco Giants': 'SFG', 'Seattle Mariners' : 'SEA', 'St. Louis Cardinals' : 'STL',\
            'Texas Rangers': 'TEX', 'Toronto Blue Jays' : 'TOR', 'Washington Nationals' : 'WSN', 'Baltimore Orioles' : 'BAL',\
            'Cleveland Guardians': 'CLE', 'Pittsburgh Pirates': 'PIT', 'Atlanta Braves' : 'ATL', 'Tampa Bay Rays': 'TBR',\
            'Detroit Tigers': 'DET', 'Los Angeles Dodgers': 'LAD', 'Chicago Cubs': 'CHC', 'Philadelphia Phillies': 'PHI',\
            'Colorado Rockies': 'COL', 'Oakland Athletics': 'OAK', 'Arizona Diamondbacks': 'ARI', 'Minnesota Twins': 'MIN',\
            'New York Yankees': 'NYY', 'San Diego Padres': 'SDP'}

In [1036]:
fanduel_df = pd.read_csv('FD_PlayersList_2022_08_31.csv')

In [1037]:
fanduel_df.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Probable Pitcher,Batting Order,Roster Position
0,79825-52859,P,Jacob,Jacob deGrom,deGrom,47.000000,5.0,12500,LAD@NYM,NYM,LAD,NaN,NaN,NaN,NaN,NaN,P
1,79825-5202,P,Justin,Justin Verlander,Verlander,41.541667,24.0,11400,HOU@TEX,HOU,TEX,DTD,Calf,NaN,NaN,NaN,P
2,79825-82604,P,Corbin,Corbin Burnes,Burnes,39.346154,26.0,11100,PIT@MIL,MIL,PIT,NaN,NaN,NaN,NaN,NaN,P
3,79825-16956,P,Gerrit,Gerrit Cole,Cole,39.500000,26.0,11000,NYY@LAA,NYY,LAA,NaN,NaN,NaN,NaN,NaN,P
4,79825-60633,P,Carlos,Carlos Rodon,Rodon,38.538462,26.0,11000,SD@SF,SF,SD,NaN,NaN,NaN,NaN,NaN,P


In [1038]:
fanduel_df = fanduel_df[['Position', 'Nickname', 'Salary', 'Team', 'Opponent']]

In [1039]:
fanduel_df.head()

,Position,Nickname,Salary,Team,Opponent
0,P,Jacob deGrom,12500,NYM,LAD
1,P,Justin Verlander,11400,HOU,TEX
2,P,Corbin Burnes,11100,MIL,PIT
3,P,Gerrit Cole,11000,NYY,LAA
4,P,Carlos Rodon,11000,SF,SD


In [1040]:
fanduel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Position  1197 non-null   object
 1   Nickname  1197 non-null   object
 2   Salary    1197 non-null   int64 
 3   Team      1197 non-null   object
 4   Opponent  1197 non-null   object
dtypes: int64(1), object(4)
memory usage: 46.9+ KB


In [1041]:
fanduel_df['Team'].value_counts()

LAA    58
MIN    56
SF     55
WSH    54
LAD    53
NYM    53
SEA    52
CHC    52
ATL    52
PIT    51
PHI    51
NYY    51
OAK    50
DET    50
ARI    50
TEX    50
KC     48
TOR    47
MIL    46
BOS    45
CWS    45
COL    44
SD     43
HOU    41
Name: Team, dtype: int64

In [1042]:
fanduel_df = fanduel_df[fanduel_df['Position'] != 'P']

In [1043]:
fanduel_df.head()

,Position,Nickname,Salary,Team,Opponent
663,OF,Aaron Judge,5000,NYY,LAA
664,OF,Mookie Betts,4600,LAD,NYM
665,OF,Yordan Alvarez,4400,HOU,TEX
666,OF,Mike Trout,4300,LAA,NYY
667,OF,Shohei Ohtani,4200,LAA,NYY


In [1044]:
len(fanduel_df)

534

In [1045]:
fanduel_df.replace({'Team': {'TB':'TBR', 'SF': 'SFG', 'SD': 'SDP', 'KC': 'KCR', 'WSH': 'WSN', 'CWS': 'CHW'},\
                    'Opponent': {'TB':'TBR', 'SF': 'SFG', 'SD': 'SDP', 'KC': 'KCR', 'WSH': 'WSN', 'CWS': 'CHW'}}, inplace = True)

In [1046]:
fanduel_df.head()

,Position,Nickname,Salary,Team,Opponent
663,OF,Aaron Judge,5000,NYY,LAA
664,OF,Mookie Betts,4600,LAD,NYM
665,OF,Yordan Alvarez,4400,HOU,TEX
666,OF,Mike Trout,4300,LAA,NYY
667,OF,Shohei Ohtani,4200,LAA,NYY


In [1047]:
#Scraping RotoGrinders MLB First Look tables
#For Batters we need the fourth table to get team, total and Handed (opp pitcher hand)
rg_string = 'https://rotogrinders.com/articles/mlb-dfs-first-look-wednesday-august-31st-sortable-odds-salaries-stats-3784479'
rg_page = requests.get(rg_string)
rg_html_doc = rg_page.text
rg_soup_obj = BeautifulSoup(rg_html_doc)

In [1048]:
#Getting all table tags
tables = rg_soup_obj.find_all('table')

In [1049]:
hitters = tables[3]

In [1050]:
hitters_headers = hitters.find_all('th')

In [1051]:
hitters_headers

[<th> 	Batter	</th>,
 <th>	Team	</th>,
 <th style="text-align:center;">	Handed	</th>,
 <th style="text-align:center;">	Order	</th>,
 <th style="text-align:center;">	Total	</th>,
 <th style="text-align:center;">	wOBA	</th>,
 <th style="text-align:center;"> <span class="caps">ISO</span> </th>,
 <th style="text-align:center;">	HC%	</th>,
 <th style="text-align:center;">	BB%	</th>,
 <th style="text-align:center;">	K%	</th>,
 <th style="text-align:center;">	DK	</th>,
 <th style="text-align:center;">	Pos	</th>,
 <th style="text-align:center;">	FD	</th>,
 <th style="text-align:center;">	Pos	</th>,
 <th>	Opponent	</th>,
 <th>	Opp Pitcher	</th>,
 <th style="text-align:center;">	Handed	</th>,
 <th style="text-align:center;">	PvsLHB	</th>,
 <th style="text-align:center;">	PvsRHB	</th>,
 <th style="text-align:center;">	Time (ET)	</th>,
 <th style="text-align:center;">	Slate	</th>]

In [1052]:
hitters_cols = []
for header in hitters_headers:
    hitters_cols.append(header.get_text().strip('\t'))

In [1053]:
hitters_cols

[' \tBatter',
 'Team',
 'Handed',
 'Order',
 'Total',
 'wOBA',
 ' ISO ',
 'HC%',
 'BB%',
 'K%',
 'DK',
 'Pos',
 'FD',
 'Pos',
 'Opponent',
 'Opp Pitcher',
 'Handed',
 'PvsLHB',
 'PvsRHB',
 'Time (ET)',
 'Slate']

In [1054]:
hitters_cols = [content.strip(' ') for content in hitters_cols]

In [1055]:
hitters_cols = [content.strip('\t') for content in hitters_cols]

In [1056]:
hitters_cols

['Batter',
 'Team',
 'Handed',
 'Order',
 'Total',
 'wOBA',
 'ISO',
 'HC%',
 'BB%',
 'K%',
 'DK',
 'Pos',
 'FD',
 'Pos',
 'Opponent',
 'Opp Pitcher',
 'Handed',
 'PvsLHB',
 'PvsRHB',
 'Time (ET)',
 'Slate']

In [1057]:
content1 = tables[3].find_all('td')

In [1058]:
content1

[<td> <a class="player-popup" href="/players/cedric-mullins-490053" id="cedric-mullins">Cedric Mullins</a> </td>,
 <td style="text-align:center;"> <span class="caps">BAL</span> </td>,
 <td style="text-align:center;">	LH	</td>,
 <td style="text-align:center;">	1	</td>,
 <td style="background:#FFD0D0; color:black; text-align:center;">	3.62	</td>,
 <td style="background:#B7D4FF; color:black; text-align:center;">	0.354	</td>,
 <td style="background:#FFD0D0; color:black; text-align:center;">	0.160	</td>,
 <td style="background:#FFFFC9; color:black; text-align:center;">	28%	</td>,
 <td style="background:#D4FFD4; color:black; text-align:center;">	8%	</td>,
 <td style="background:#B7D4FF; color:black; text-align:center;">	15%	</td>,
 <td style="text-align:center;">	$5,300	</td>,
 <td style="text-align:center;">	OF	</td>,
 <td style="text-align:center;">	$3,100	</td>,
 <td style="text-align:center;">	OF	</td>,
 <td>	at <span class="caps">CLE</span> </td>,
 <td> <a class="player-popup" href="/pl

In [1059]:
hitters_table_content = []
for item in content1:
    hitters_table_content.append(item.get_text().strip(' '))

In [1060]:
hitters_table_content

['Cedric Mullins',
 'BAL',
 '\tLH\t',
 '\t1\t',
 '\t3.62\t',
 '\t0.354\t',
 '\t0.160\t',
 '\t28%\t',
 '\t8%\t',
 '\t15%\t',
 '\t$5,300\t',
 '\tOF\t',
 '\t$3,100\t',
 '\tOF\t',
 '\tat CLE',
 'Cal Quantrill',
 'RHP',
 '\t0.312\t',
 '\t0.304\t',
 '\t6:10 \t',
 '',
 'Adley Rutschman',
 'BAL',
 '\tSH\t',
 '\t2\t',
 '\t3.62\t',
 '\t0.388\t',
 '\t0.227\t',
 '\t27%\t',
 '\t13%\t',
 '\t15%\t',
 '\t$5,500\t',
 '\tC\t',
 '\t$3,000\t',
 '\tC\t',
 '\tat CLE',
 'Cal Quantrill',
 'RHP',
 '\t0.312\t',
 '\t0.304\t',
 '\t6:10 \t',
 '',
 'Anthony Santander',
 'BAL',
 '\tSH\t',
 '\t3\t',
 '\t3.62\t',
 '\t0.327\t',
 '\t0.183\t',
 '\t34%\t',
 '\t8%\t',
 '\t17%\t',
 '\t$5,100\t',
 '\tOF\t',
 '\t$2,900\t',
 '\tOF\t',
 '\tat CLE',
 'Cal Quantrill',
 'RHP',
 '\t0.312\t',
 '\t0.304\t',
 '\t6:10 \t',
 '',
 'Ryan Mountcastle',
 'BAL',
 '\tRH\t',
 '\t4\t',
 '\t3.62\t',
 '\t0.302\t',
 '\t0.170\t',
 '\t33%\t',
 '\t6%\t',
 '\t27%\t',
 '\t$3,900\t',
 '\t1B\t',
 '\t$2,800\t',
 '\t1B\t',
 '\tat CLE',
 'Cal Quantrill',
 '

In [1061]:
hitters_table_content = [item.strip('\t') for item in hitters_table_content]

In [1062]:
hitters_table_content

['Cedric Mullins',
 'BAL',
 'LH',
 '1',
 '3.62',
 '0.354',
 '0.160',
 '28%',
 '8%',
 '15%',
 '$5,300',
 'OF',
 '$3,100',
 'OF',
 'at CLE',
 'Cal Quantrill',
 'RHP',
 '0.312',
 '0.304',
 '6:10 ',
 '',
 'Adley Rutschman',
 'BAL',
 'SH',
 '2',
 '3.62',
 '0.388',
 '0.227',
 '27%',
 '13%',
 '15%',
 '$5,500',
 'C',
 '$3,000',
 'C',
 'at CLE',
 'Cal Quantrill',
 'RHP',
 '0.312',
 '0.304',
 '6:10 ',
 '',
 'Anthony Santander',
 'BAL',
 'SH',
 '3',
 '3.62',
 '0.327',
 '0.183',
 '34%',
 '8%',
 '17%',
 '$5,100',
 'OF',
 '$2,900',
 'OF',
 'at CLE',
 'Cal Quantrill',
 'RHP',
 '0.312',
 '0.304',
 '6:10 ',
 '',
 'Ryan Mountcastle',
 'BAL',
 'RH',
 '4',
 '3.62',
 '0.302',
 '0.170',
 '33%',
 '6%',
 '27%',
 '$3,900',
 '1B',
 '$2,800',
 '1B',
 'at CLE',
 'Cal Quantrill',
 'RHP',
 '0.312',
 '0.304',
 '6:10 ',
 '',
 'Austin Hays',
 'BAL',
 'RH',
 '5',
 '3.62',
 '0.330',
 '0.183',
 '33%',
 '6%',
 '20%',
 '$3,300',
 'OF',
 '$2,700',
 'OF',
 'at CLE',
 'Cal Quantrill',
 'RHP',
 '0.312',
 '0.304',
 '6:10 ',
 ''

In [1063]:
num_rows = len(hitters_table_content)/len(hitters_cols)

In [1064]:
hitters_table_content = iter(hitters_table_content)

In [1065]:
hitters_rows = []
for i in range(int(num_rows)):
    new_row = []
    for j in range(len(hitters_cols)):
        new_row.append(next(hitters_table_content))
    hitters_rows.append(new_row)

In [1066]:
hitters_rows

[['Cedric Mullins',
  'BAL',
  'LH',
  '1',
  '3.62',
  '0.354',
  '0.160',
  '28%',
  '8%',
  '15%',
  '$5,300',
  'OF',
  '$3,100',
  'OF',
  'at CLE',
  'Cal Quantrill',
  'RHP',
  '0.312',
  '0.304',
  '6:10 ',
  ''],
 ['Adley Rutschman',
  'BAL',
  'SH',
  '2',
  '3.62',
  '0.388',
  '0.227',
  '27%',
  '13%',
  '15%',
  '$5,500',
  'C',
  '$3,000',
  'C',
  'at CLE',
  'Cal Quantrill',
  'RHP',
  '0.312',
  '0.304',
  '6:10 ',
  ''],
 ['Anthony Santander',
  'BAL',
  'SH',
  '3',
  '3.62',
  '0.327',
  '0.183',
  '34%',
  '8%',
  '17%',
  '$5,100',
  'OF',
  '$2,900',
  'OF',
  'at CLE',
  'Cal Quantrill',
  'RHP',
  '0.312',
  '0.304',
  '6:10 ',
  ''],
 ['Ryan Mountcastle',
  'BAL',
  'RH',
  '4',
  '3.62',
  '0.302',
  '0.170',
  '33%',
  '6%',
  '27%',
  '$3,900',
  '1B',
  '$2,800',
  '1B',
  'at CLE',
  'Cal Quantrill',
  'RHP',
  '0.312',
  '0.304',
  '6:10 ',
  ''],
 ['Austin Hays',
  'BAL',
  'RH',
  '5',
  '3.62',
  '0.330',
  '0.183',
  '33%',
  '6%',
  '20%',
  '$3,30

In [1067]:
hitters_df = pd.DataFrame(hitters_rows, columns = hitters_cols)

In [1068]:
hitters_df.head()

,Batter,Team,Handed,Order,Total,wOBA,ISO,HC%,BB%,K%,...,Pos,FD,Pos,Opponent,Opp Pitcher,Handed,PvsLHB,PvsRHB,Time (ET),Slate
0,Cedric Mullins,BAL,LH,1,3.62,0.354,0.160,28%,8%,15%,...,OF,"$3,100",OF,at CLE,Cal Quantrill,RHP,0.312,0.304,6:10,
1,Adley Rutschman,BAL,SH,2,3.62,0.388,0.227,27%,13%,15%,...,C,"$3,000",C,at CLE,Cal Quantrill,RHP,0.312,0.304,6:10,
2,Anthony Santander,BAL,SH,3,3.62,0.327,0.183,34%,8%,17%,...,OF,"$2,900",OF,at CLE,Cal Quantrill,RHP,0.312,0.304,6:10,
3,Ryan Mountcastle,BAL,RH,4,3.62,0.302,0.170,33%,6%,27%,...,1B,"$2,800",1B,at CLE,Cal Quantrill,RHP,0.312,0.304,6:10,
4,Austin Hays,BAL,RH,5,3.62,0.330,0.183,33%,6%,20%,...,OF,"$2,700",OF,at CLE,Cal Quantrill,RHP,0.312,0.304,6:10,


In [1069]:
hitters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Batter       270 non-null    object
 1   Team         270 non-null    object
 2   Handed       270 non-null    object
 3   Order        270 non-null    object
 4   Total        270 non-null    object
 5   wOBA         270 non-null    object
 6   ISO          270 non-null    object
 7   HC%          270 non-null    object
 8   BB%          270 non-null    object
 9   K%           270 non-null    object
 10  DK           270 non-null    object
 11  Pos          270 non-null    object
 12  FD           270 non-null    object
 13  Pos          270 non-null    object
 14  Opponent     270 non-null    object
 15  Opp Pitcher  270 non-null    object
 16  Handed       270 non-null    object
 17  PvsLHB       270 non-null    object
 18  PvsRHB       270 non-null    object
 19  Time (ET)    270 non-null    

In [1070]:
hitters_df['Team'].value_counts()

CHC    9
TEX    9
SEA    9
PIT    9
MIA    9
NYM    9
MIL    9
COL    9
SFG    9
WAS    9
OAK    9
HOU    9
ARI    9
CHW    9
ATL    9
CIN    9
KCR    9
NYY    9
PHI    9
BOS    9
BAL    9
MIN    9
LAA    9
TOR    9
LAD    9
STL    9
TBR    9
SDP    9
DET    9
CLE    9
Name: Team, dtype: int64

In [1071]:
hitters_df = hitters_df.iloc[:, [0, 1, 4, 14, 16]]

In [1072]:
hitters_df.head()

,Batter,Team,Total,Opponent,Handed
0,Cedric Mullins,BAL,3.62,at CLE,RHP
1,Adley Rutschman,BAL,3.62,at CLE,RHP
2,Anthony Santander,BAL,3.62,at CLE,RHP
3,Ryan Mountcastle,BAL,3.62,at CLE,RHP
4,Austin Hays,BAL,3.62,at CLE,RHP


In [1073]:
#hitters_df['Offense'] = hitters_df['Offense'].replace(team_dict)

In [1074]:
hitters_df['Park'] = np.where(hitters_df['Opponent'].str[0] == 'a', hitters_df['Opponent'].str[3:], hitters_df['Team'])

In [1075]:
hitters_df

,Batter,Team,Total,Opponent,Handed,Park
0,Cedric Mullins,BAL,3.62,at CLE,RHP,CLE
1,Adley Rutschman,BAL,3.62,at CLE,RHP,CLE
2,Anthony Santander,BAL,3.62,at CLE,RHP,CLE
3,Ryan Mountcastle,BAL,3.62,at CLE,RHP,CLE
4,Austin Hays,BAL,3.62,at CLE,RHP,CLE
...,...,...,...,...,...,...
265,Thairo Estrada,SFG,3.58,vs SDP,LHP,SFG
266,Brandon Crawford,SFG,3.58,vs SDP,LHP,SFG
267,Joey Bart,SFG,3.58,vs SDP,LHP,SFG
268,Austin Wynns,SFG,3.58,vs SDP,LHP,SFG


In [1076]:
hitters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Batter    270 non-null    object
 1   Team      270 non-null    object
 2   Total     270 non-null    object
 3   Opponent  270 non-null    object
 4   Handed    270 non-null    object
 5   Park      270 non-null    object
dtypes: object(6)
memory usage: 12.8+ KB


In [1077]:
#hitters_df.rename(columns = {'Offense':'Team'}, inplace = True)

In [1078]:
hitters_df['Opponent'] = hitters_df['Opponent'].str[-3:]

In [1079]:
hitters_df.rename(columns = {'Handed': 'Opp_Hand'}, inplace = True)

In [1080]:
hitters_df.drop(columns = 'Batter', inplace = True)

In [1081]:
hitters_df = hitters_df.drop_duplicates(keep = 'last')

In [1082]:
hitters_df = hitters_df.replace('WAS', 'WSN')

In [1083]:
main_df = pd.merge(fanduel_df, hitters_df, on = ['Team', 'Opponent'], how = 'left')

In [1084]:
null_data = main_df[main_df.isnull().any(axis=1)]

In [1085]:
null_data['Team'].value_counts()

Series([], Name: Team, dtype: int64)

In [1086]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 0 to 533
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Position  534 non-null    object
 1   Nickname  534 non-null    object
 2   Salary    534 non-null    int64 
 3   Team      534 non-null    object
 4   Opponent  534 non-null    object
 5   Total     534 non-null    object
 6   Opp_Hand  534 non-null    object
 7   Park      534 non-null    object
dtypes: int64(1), object(7)
memory usage: 37.5+ KB


In [1087]:
opp_df = hitters_df[['Team', 'Total']].rename(columns = {'Team': 'Opponent', 'Total': 'Opp_Total'})

In [1088]:
opp_df = opp_df.drop_duplicates(keep = 'last')

In [1089]:
opp_df

,Opponent,Opp_Total
8,BAL,3.62
17,CLE,4.38
26,STL,6.30
35,CIN,4.20
44,TBR,3.83
53,MIA,2.67
62,OAK,4.25
71,WSN,4.25
80,CHC,3.02
89,TOR,4.48


In [1091]:
main_df = pd.merge(main_df, opp_df, on = 'Opponent', how = 'left')

In [1092]:
main_df.head()

,Position,Nickname,Salary,Team,Opponent,Total,Opp_Hand,Park,Opp_Total
0,OF,Aaron Judge,5000,NYY,LAA,4.75,RHP,LAA,3.75
1,OF,Mookie Betts,4600,LAD,NYM,4.55,RHP,NYM,3.95
2,OF,Yordan Alvarez,4400,HOU,TEX,4.21,RHP,TEX,3.29
3,OF,Mike Trout,4300,LAA,NYY,3.75,RHP,LAA,4.75
4,OF,Shohei Ohtani,4200,LAA,NYY,3.75,RHP,LAA,4.75


In [1093]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 0 to 533
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Position   534 non-null    object
 1   Nickname   534 non-null    object
 2   Salary     534 non-null    int64 
 3   Team       534 non-null    object
 4   Opponent   534 non-null    object
 5   Total      534 non-null    object
 6   Opp_Hand   534 non-null    object
 7   Park       534 non-null    object
 8   Opp_Total  534 non-null    object
dtypes: int64(1), object(8)
memory usage: 41.7+ KB


In [1094]:
main_df[['Total', 'Opp_Total']] = main_df[['Total', 'Opp_Total']].astype('float')

In [1095]:
main_df['OpenOU'] = main_df['Total'] + main_df['Opp_Total']

In [1096]:
main_df.head()

,Position,Nickname,Salary,Team,Opponent,Total,Opp_Hand,Park,Opp_Total,OpenOU
0,OF,Aaron Judge,5000,NYY,LAA,4.75,RHP,LAA,3.75,8.5
1,OF,Mookie Betts,4600,LAD,NYM,4.55,RHP,NYM,3.95,8.5
2,OF,Yordan Alvarez,4400,HOU,TEX,4.21,RHP,TEX,3.29,7.5
3,OF,Mike Trout,4300,LAA,NYY,3.75,RHP,LAA,4.75,8.5
4,OF,Shohei Ohtani,4200,LAA,NYY,3.75,RHP,LAA,4.75,8.5


In [1097]:
main_df.loc[:, 'Opp_Hand'] = main_df.loc[:, 'Opp_Hand'].str[0]

In [1098]:
main_df.head()

,Position,Nickname,Salary,Team,Opponent,Total,Opp_Hand,Park,Opp_Total,OpenOU
0,OF,Aaron Judge,5000,NYY,LAA,4.75,R,LAA,3.75,8.5
1,OF,Mookie Betts,4600,LAD,NYM,4.55,R,NYM,3.95,8.5
2,OF,Yordan Alvarez,4400,HOU,TEX,4.21,R,TEX,3.29,7.5
3,OF,Mike Trout,4300,LAA,NYY,3.75,R,LAA,4.75,8.5
4,OF,Shohei Ohtani,4200,LAA,NYY,3.75,R,LAA,4.75,8.5


In [1099]:
main_df.drop(columns = ['Opp_Total'], inplace = True)

In [1100]:
main_df.rename(columns = {'Nickname':'Name', 'Opp_Hand': 'Hand'}, inplace = True)

In [1101]:
len(main_df)

534

These are the columns from Ind Batting L30 that we kept
cols_to_keep = ['Name', 'Team', 'Events', 'EV', 'maxEV', 'LA', 'Barrels', 'Barrel%', 'HardHit', 'HardHit%', 'AVG', 'wOBA', 'BB%', 'K%', 'OPS', 'ISO', 'BABIP', 'GB/FB', 'LD%', 'HR/FB',  'Soft%', 'Med%', 'Hard%', 'Date']

In [1102]:
today = dt.date.today()

In [1103]:
today_str = str(today).replace('-', '_')

In [1104]:
today_str

'2022_08_30'

# Must run the batting scraping notebook before running the next cell

In [1105]:
ind_L30_df = pd.read_csv('IndividualBatting_L30_{}.csv'.format(today_str))

In [1106]:
ind_L30_df[ind_L30_df['Name'] == 'Aaron Judge']

,Unnamed: 0,Name,Team,PA,Events,EV,maxEV,LA,Barrels,Barrel%,...,F-Strike%,SwStr%,Soft%,Med%,Hard%,Pull%,Cent%,Oppo%,wRC+,Date
21,21,Aaron Judge,NYY,120,64,94.7,116.9,18.3,13,20.3%,...,56.7%,11.8%,7.9%,49.2%,42.9%,36.5%,34.9%,28.6%,205,2022-08-30


In [1107]:
ind_L30_df = ind_L30_df[['Name', 'Team', 'Date', 'Barrel%', 'Barrels', 'EV', 'Events', 'HR/FB', 'Hard%', 'HardHit',\
                         'HardHit%', 'ISO', 'Med%', 'OPS', 'maxEV', 'wOBA']]

In [1108]:
ind_L30_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      505 non-null    object 
 1   Team      505 non-null    object 
 2   Date      505 non-null    object 
 3   Barrel%   505 non-null    object 
 4   Barrels   505 non-null    object 
 5   EV        505 non-null    object 
 6   Events    505 non-null    int64  
 7   HR/FB     505 non-null    object 
 8   Hard%     505 non-null    object 
 9   HardHit   505 non-null    object 
 10  HardHit%  505 non-null    object 
 11  ISO       505 non-null    float64
 12  Med%      505 non-null    object 
 13  OPS       505 non-null    float64
 14  maxEV     505 non-null    object 
 15  wOBA      505 non-null    float64
dtypes: float64(3), int64(1), object(12)
memory usage: 63.2+ KB


In [1109]:
ind_L30_df[ind_L30_df['Name'] == 'Bryce Harper']

,Name,Team,Date,Barrel%,Barrels,EV,Events,HR/FB,Hard%,HardHit,HardHit%,ISO,Med%,OPS,maxEV,wOBA
65,Bryce Harper,PHI,2022-08-30,15.4%,2,92.1,13,0.0%,38.5%,6,46.2%,0.143,61.5%,0.971,113.6,0.391


In [1110]:
main_df = pd.merge(main_df, ind_L30_df, on = ['Name'], how = 'left', suffixes = [None, '_Season'])

In [1111]:
main_df.head()

,Position,Name,Salary,Team,Opponent,Total,Hand,Park,OpenOU,Team_Season,...,Events,HR/FB,Hard%,HardHit,HardHit%,ISO,Med%,OPS,maxEV,wOBA
0,OF,Aaron Judge,5000,NYY,LAA,4.75,R,LAA,8.5,NYY,...,64.0,27.3%,42.9%,37,57.8%,0.348,49.2%,1.091,116.9,0.453
1,OF,Mookie Betts,4600,LAD,NYM,4.55,R,NYM,8.5,LAD,...,94.0,18.2%,44.7%,46,48.9%,0.324,45.7%,0.998,109.0,0.421
2,OF,Yordan Alvarez,4400,HOU,TEX,4.21,R,TEX,7.5,HOU,...,65.0,4.8%,41.5%,38,58.5%,0.070,46.2%,0.604,114.6,0.276
3,OF,Mike Trout,4300,LAA,NYY,3.75,R,LAA,8.5,LAA,...,28.0,28.6%,39.3%,10,35.7%,0.333,39.3%,0.998,113.0,0.425
4,OF,Shohei Ohtani,4200,LAA,NYY,3.75,R,LAA,8.5,LAA,...,69.0,26.7%,44.9%,32,46.4%,0.354,44.9%,1.085,110.7,0.447


In [1112]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 0 to 533
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Position     534 non-null    object 
 1   Name         534 non-null    object 
 2   Salary       534 non-null    int64  
 3   Team         534 non-null    object 
 4   Opponent     534 non-null    object 
 5   Total        534 non-null    float64
 6   Hand         534 non-null    object 
 7   Park         534 non-null    object 
 8   OpenOU       534 non-null    float64
 9   Team_Season  402 non-null    object 
 10  Date         402 non-null    object 
 11  Barrel%      402 non-null    object 
 12  Barrels      402 non-null    object 
 13  EV           402 non-null    object 
 14  Events       402 non-null    float64
 15  HR/FB        402 non-null    object 
 16  Hard%        402 non-null    object 
 17  HardHit      402 non-null    object 
 18  HardHit%     402 non-null    object 
 19  ISO     

In [1113]:
hands = ['LHP', 'RHP']

In [1114]:
df_vs_LHP = pd.read_csv('IndividualBattingVs_LHP_{}.csv'.format(today_str))
df_vs_RHP = pd.read_csv('IndividualBattingVs_RHP_{}.csv'.format(today_str))

In [1115]:
df_vs_LHP['Hand'] = 'L'
df_vs_RHP['Hand'] = 'R'

In [1116]:
hands_df = pd.concat([df_vs_LHP, df_vs_RHP])

In [1117]:
hands_df.shape

(1274, 26)

In [1118]:
df_vs_LHP.shape

(627, 26)

In [1119]:
hands_df = hands_df[['Name', 'Hand', 'HR/FB', 'Hard%', 'ISO', 'Med%', 'OPS', 'wOBA']]

In [1120]:
hands_df.head()

,Name,Hand,HR/FB,Hard%,ISO,Med%,OPS,wOBA
0,Chadwick Tromp,L,0.0%,33.3%,0.667,66.7%,2.667,1.136
1,Joe Dunand,L,50.0%,66.7%,1.000,33.3%,2.000,0.833
2,Matt Beaty,L,0.0%,0.0%,1.000,100.0%,2.000,0.801
3,Kyle Stowers,L,0.0%,,0.000,,1.000,0.721
4,Jason Krizan,L,0.0%,,0.000,,1.000,0.690


In [1121]:
main_df = pd.merge(main_df, hands_df, on = ['Name', 'Hand'], how = 'left', suffixes = [None, 'vs_Hand'])

In [1122]:
main_df.head()

,Position,Name,Salary,Team,Opponent,Total,Hand,Park,OpenOU,Team_Season,...,Med%,OPS,maxEV,wOBA,HR/FBvs_Hand,Hard%vs_Hand,ISOvs_Hand,Med%vs_Hand,OPSvs_Hand,wOBAvs_Hand
0,OF,Aaron Judge,5000,NYY,LAA,4.75,R,LAA,8.5,NYY,...,49.2%,1.091,116.9,0.453,32.5%,45.2%,0.363,47.2%,1.065,0.446
1,OF,Mookie Betts,4600,LAD,NYM,4.55,R,NYM,8.5,LAD,...,45.7%,0.998,109.0,0.421,18.2%,37.6%,0.268,50.0%,0.867,0.371
2,OF,Yordan Alvarez,4400,HOU,TEX,4.21,R,TEX,7.5,HOU,...,46.2%,0.604,114.6,0.276,28.2%,45.8%,0.321,43.8%,1.027,0.427
3,OF,Mike Trout,4300,LAA,NYY,3.75,R,LAA,8.5,LAA,...,39.3%,0.998,113.0,0.425,23.7%,37.8%,0.357,47.4%,0.958,0.401
4,OF,Shohei Ohtani,4200,LAA,NYY,3.75,R,LAA,8.5,LAA,...,44.9%,1.085,110.7,0.447,25.0%,38.7%,0.292,54.1%,0.945,0.395


In [1123]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 0 to 533
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Position      534 non-null    object 
 1   Name          534 non-null    object 
 2   Salary        534 non-null    int64  
 3   Team          534 non-null    object 
 4   Opponent      534 non-null    object 
 5   Total         534 non-null    float64
 6   Hand          534 non-null    object 
 7   Park          534 non-null    object 
 8   OpenOU        534 non-null    float64
 9   Team_Season   402 non-null    object 
 10  Date          402 non-null    object 
 11  Barrel%       402 non-null    object 
 12  Barrels       402 non-null    object 
 13  EV            402 non-null    object 
 14  Events        402 non-null    float64
 15  HR/FB         402 non-null    object 
 16  Hard%         402 non-null    object 
 17  HardHit       402 non-null    object 
 18  HardHit%      402 non-null    

# Scrape Temp 

In [1126]:
temp_df = pd.read_csv('TempWind_2022_08_30.csv')

In [1127]:
temp_df

,Unnamed: 0,Date,Park,Temp,Wind
0,0,2022-08-30,CLE,80,3
1,1,2022-08-30,MIA,85,1
2,2,2022-08-30,CIN,83,5
3,3,2022-08-30,WSN,79,3
4,4,2022-08-30,TOR,75,3
5,5,2022-08-30,NYM,80,5
6,6,2022-08-30,DET,78,3
7,7,2022-08-30,ATL,84,4
8,8,2022-08-30,MIN,80,4
9,9,2022-08-30,TEX,84,3


Check to make sure which indoor parks have games today

In [1128]:
temp_df['Temp'] = np.where(temp_df['Park'].isin(['MIA', 'ARI', 'TEX']), 72, temp_df['Temp'])

In [1129]:
temp_df

,Unnamed: 0,Date,Park,Temp,Wind
0,0,2022-08-30,CLE,80,3
1,1,2022-08-30,MIA,72,1
2,2,2022-08-30,CIN,83,5
3,3,2022-08-30,WSN,79,3
4,4,2022-08-30,TOR,75,3
5,5,2022-08-30,NYM,80,5
6,6,2022-08-30,DET,78,3
7,7,2022-08-30,ATL,84,4
8,8,2022-08-30,MIN,80,4
9,9,2022-08-30,TEX,72,3


In [1130]:
temp_df = temp_df[['Park', 'Temp']]

In [1131]:
main_df = pd.merge(main_df, temp_df, on = 'Park', how = 'left')

In [1132]:
main_df.head()

,Position,Name,Salary,Team,Opponent,Total,Hand,Park,OpenOU,Team_Season,...,OPS,maxEV,wOBA,HR/FBvs_Hand,Hard%vs_Hand,ISOvs_Hand,Med%vs_Hand,OPSvs_Hand,wOBAvs_Hand,Temp
0,OF,Aaron Judge,5000,NYY,LAA,4.75,R,LAA,8.5,NYY,...,1.091,116.9,0.453,32.5%,45.2%,0.363,47.2%,1.065,0.446,86
1,OF,Mookie Betts,4600,LAD,NYM,4.55,R,NYM,8.5,LAD,...,0.998,109.0,0.421,18.2%,37.6%,0.268,50.0%,0.867,0.371,80
2,OF,Yordan Alvarez,4400,HOU,TEX,4.21,R,TEX,7.5,HOU,...,0.604,114.6,0.276,28.2%,45.8%,0.321,43.8%,1.027,0.427,72
3,OF,Mike Trout,4300,LAA,NYY,3.75,R,LAA,8.5,LAA,...,0.998,113.0,0.425,23.7%,37.8%,0.357,47.4%,0.958,0.401,86
4,OF,Shohei Ohtani,4200,LAA,NYY,3.75,R,LAA,8.5,LAA,...,1.085,110.7,0.447,25.0%,38.7%,0.292,54.1%,0.945,0.395,86


In [1133]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 0 to 533
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Position      534 non-null    object 
 1   Name          534 non-null    object 
 2   Salary        534 non-null    int64  
 3   Team          534 non-null    object 
 4   Opponent      534 non-null    object 
 5   Total         534 non-null    float64
 6   Hand          534 non-null    object 
 7   Park          534 non-null    object 
 8   OpenOU        534 non-null    float64
 9   Team_Season   402 non-null    object 
 10  Date          402 non-null    object 
 11  Barrel%       402 non-null    object 
 12  Barrels       402 non-null    object 
 13  EV            402 non-null    object 
 14  Events        402 non-null    float64
 15  HR/FB         402 non-null    object 
 16  Hard%         402 non-null    object 
 17  HardHit       402 non-null    object 
 18  HardHit%      402 non-null    

In [1134]:
park_df = pd.read_csv('Park_Factors_Pitch.csv')

In [1135]:
park_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Team,Venue,Park Factor,HR,SO
1,Rockies,Coors Field,111,113,84
2,Reds,Great American Ball Park,111,153,105
3,Red Sox,Fenway Park,109,101,94
4,Angels,Angel Stadium,105,119,98


In [1136]:
park_df = park_df.iloc[1:, :]

In [1137]:
park_df.columns = ['Team', 'Venue', 'Pk_Fct_Overall', 'Pk_Fct_HR', 'Pk_Fct_SO']

In [1138]:
list(park_df['Team'])

['Rockies',
 'Reds',
 'Red Sox',
 'Angels',
 'Royals',
 'Braves',
 'Phillies',
 'Orioles',
 'White Sox',
 'Blue Jays',
 'D-backs',
 'Dodgers',
 'Pirates',
 'Guardians',
 'Brewers',
 'Giants',
 'Cubs',
 'Nationals',
 'Yankees',
 'Marlins',
 'Astros',
 'Rangers',
 'Mets',
 'Tigers',
 'Twins',
 'Rays',
 'Cardinals',
 'Athletics',
 'Padres',
 'Mariners']

In [1139]:
park_team_dict = {'Rockies': 'COL', 'Reds': 'CIN', 'Red Sox': 'BOS', 'Angels': 'LAA', 'Royals': 'KCR', 'Braves': 'ATL',\
                 'Phillies': 'PHI', 'Orioles': 'BAL', 'White Sox': 'CHW', 'Blue Jays': 'TOR', 'D-backs': 'ARI',\
                 'Dodgers': 'LAD', 'Pirates': 'PIT', 'Guardians': 'CLE', 'Brewers': 'MIL', 'Giants': 'SFG', 'Cubs': 'CHC',\
                 'Nationals': 'WSN', 'Yankees': 'NYY', 'Marlins': 'MIA', 'Astros': 'HOU', 'Rangers': 'TEX', 'Mets': 'NYM',\
                 'Tigers': 'DET', 'Twins': 'MIN', 'Rays': 'TBR', 'Cardinals': 'STL', 'Athletics': 'OAK', 'Padres': 'SDP',\
                 'Mariners': 'SEA'}

In [1140]:
park_df.rename(columns = {'Team': 'Park'}, inplace = True)

In [1141]:
park_df.drop(columns = 'Venue', inplace = True)

In [1142]:
park_df['Park'].replace(park_team_dict, inplace = True)

In [1143]:
park_df = park_df[['Park', 'Pk_Fct_Overall']]

In [1144]:
park_df

,Park,Pk_Fct_Overall
1,COL,111
2,CIN,111
3,BOS,109
4,LAA,105
5,KCR,102
6,ATL,102
7,PHI,102
8,BAL,101
9,CHW,101
10,TOR,101


In [1145]:
main_df = pd.merge(main_df, park_df, on = 'Park', how = 'left')

In [1146]:
main_df.head()

,Position,Name,Salary,Team,Opponent,Total,Hand,Park,OpenOU,Team_Season,...,maxEV,wOBA,HR/FBvs_Hand,Hard%vs_Hand,ISOvs_Hand,Med%vs_Hand,OPSvs_Hand,wOBAvs_Hand,Temp,Pk_Fct_Overall
0,OF,Aaron Judge,5000,NYY,LAA,4.75,R,LAA,8.5,NYY,...,116.9,0.453,32.5%,45.2%,0.363,47.2%,1.065,0.446,86,105
1,OF,Mookie Betts,4600,LAD,NYM,4.55,R,NYM,8.5,LAD,...,109.0,0.421,18.2%,37.6%,0.268,50.0%,0.867,0.371,80,98
2,OF,Yordan Alvarez,4400,HOU,TEX,4.21,R,TEX,7.5,HOU,...,114.6,0.276,28.2%,45.8%,0.321,43.8%,1.027,0.427,72,98
3,OF,Mike Trout,4300,LAA,NYY,3.75,R,LAA,8.5,LAA,...,113.0,0.425,23.7%,37.8%,0.357,47.4%,0.958,0.401,86,105
4,OF,Shohei Ohtani,4200,LAA,NYY,3.75,R,LAA,8.5,LAA,...,110.7,0.447,25.0%,38.7%,0.292,54.1%,0.945,0.395,86,105


In [1147]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 0 to 533
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Position        534 non-null    object 
 1   Name            534 non-null    object 
 2   Salary          534 non-null    int64  
 3   Team            534 non-null    object 
 4   Opponent        534 non-null    object 
 5   Total           534 non-null    float64
 6   Hand            534 non-null    object 
 7   Park            534 non-null    object 
 8   OpenOU          534 non-null    float64
 9   Team_Season     402 non-null    object 
 10  Date            402 non-null    object 
 11  Barrel%         402 non-null    object 
 12  Barrels         402 non-null    object 
 13  EV              402 non-null    object 
 14  Events          402 non-null    float64
 15  HR/FB           402 non-null    object 
 16  Hard%           402 non-null    object 
 17  HardHit         402 non-null    obj

In [1148]:
obj_cols = list(main_df.select_dtypes(include = object).columns)

In [1149]:
obj_cols

['Position',
 'Name',
 'Team',
 'Opponent',
 'Hand',
 'Park',
 'Team_Season',
 'Date',
 'Barrel%',
 'Barrels',
 'EV',
 'HR/FB',
 'Hard%',
 'HardHit',
 'HardHit%',
 'Med%',
 'maxEV',
 'HR/FBvs_Hand',
 'Hard%vs_Hand',
 'Med%vs_Hand',
 'Pk_Fct_Overall']

In [1150]:
cols_to_remove = ['Position', 'Name', 'Team', 'Opponent', 'Hand', 'Park', 'Team_Season', 'Date']

In [1151]:
for col in cols_to_remove:
    obj_cols.remove(col)

In [1152]:
obj_cols

['Barrel%',
 'Barrels',
 'EV',
 'HR/FB',
 'Hard%',
 'HardHit',
 'HardHit%',
 'Med%',
 'maxEV',
 'HR/FBvs_Hand',
 'Hard%vs_Hand',
 'Med%vs_Hand',
 'Pk_Fct_Overall']

In [1153]:
main_df = main_df.replace({'\%':''}, regex = True)

In [1154]:
main_df = main_df.replace(r'^\s*$', np.nan, regex=True)

In [1155]:
def change_to_num(cols, df):
    for col in cols:
        df[col] = pd.to_numeric(df[col]) 

In [1156]:
change_to_num(obj_cols, main_df)

In [1157]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 0 to 533
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Position        534 non-null    object 
 1   Name            534 non-null    object 
 2   Salary          534 non-null    int64  
 3   Team            534 non-null    object 
 4   Opponent        534 non-null    object 
 5   Total           534 non-null    float64
 6   Hand            534 non-null    object 
 7   Park            534 non-null    object 
 8   OpenOU          534 non-null    float64
 9   Team_Season     402 non-null    object 
 10  Date            402 non-null    object 
 11  Barrel%         397 non-null    float64
 12  Barrels         400 non-null    float64
 13  EV              395 non-null    float64
 14  Events          402 non-null    float64
 15  HR/FB           402 non-null    float64
 16  Hard%           397 non-null    float64
 17  HardHit         400 non-null    flo

In [1158]:
main_df.rename(columns = {'Total':'Proj_Runs'}, inplace = True)

In [1159]:
main_df.drop(columns = ['Team_Season', 'Park', 'Hand', 'Opponent'], inplace = True)

In [1160]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 0 to 533
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Position        534 non-null    object 
 1   Name            534 non-null    object 
 2   Salary          534 non-null    int64  
 3   Team            534 non-null    object 
 4   Proj_Runs       534 non-null    float64
 5   OpenOU          534 non-null    float64
 6   Date            402 non-null    object 
 7   Barrel%         397 non-null    float64
 8   Barrels         400 non-null    float64
 9   EV              395 non-null    float64
 10  Events          402 non-null    float64
 11  HR/FB           402 non-null    float64
 12  Hard%           397 non-null    float64
 13  HardHit         400 non-null    float64
 14  HardHit%        397 non-null    float64
 15  ISO             402 non-null    float64
 16  Med%            397 non-null    float64
 17  OPS             402 non-null    flo

In [1161]:
main_df['Date'] = today

In [1162]:
cols_with_missing = list(main_df[main_df.columns[main_df.isna().any()]].columns)

In [1163]:
cols_with_missing

['Barrel%',
 'Barrels',
 'EV',
 'Events',
 'HR/FB',
 'Hard%',
 'HardHit',
 'HardHit%',
 'ISO',
 'Med%',
 'OPS',
 'maxEV',
 'wOBA',
 'HR/FBvs_Hand',
 'Hard%vs_Hand',
 'ISOvs_Hand',
 'Med%vs_Hand',
 'OPSvs_Hand',
 'wOBAvs_Hand']

In [1164]:
null_data = main_df[main_df.isnull().any(axis=1)]

In [1165]:
null_data['Salary'].value_counts()

2000    131
2500      3
2200      2
2700      2
2300      1
2900      1
2600      1
Name: Salary, dtype: int64

In [1166]:
df_2500 = main_df[main_df['Salary'] == 2000]

In [1167]:
df_2500 = df_2500.select_dtypes(include = ['int64', 'float64'])

In [2579]:
main_df['ISO'].mean()

0.11899999999999998

In [1168]:
fill_values = dict(np.round(df_2500.mean(axis = 0, skipna = True), 3))

In [1169]:
fill_values

{'Salary': 2103.675,
 'Proj_Runs': 4.021,
 'OpenOU': 8.072,
 'Barrel%': 4.796,
 'Barrels': 1.55,
 'EV': 87.296,
 'Events': 28.719,
 'HR/FB': 6.818,
 'Hard%': 27.02,
 'HardHit': 9.482,
 'HardHit%': 31.848,
 'ISO': 0.099,
 'Med%': 56.034,
 'OPS': 0.558,
 'maxEV': 104.438,
 'wOBA': 0.249,
 'HR/FBvs_Hand': 7.255,
 'Hard%vs_Hand': 26.462,
 'ISOvs_Hand': 0.101,
 'Med%vs_Hand': 55.588,
 'OPSvs_Hand': 0.551,
 'wOBAvs_Hand': 0.247,
 'Temp': 76.024,
 'Pk_Fct_Overall': 99.966}

In [1170]:
for col in cols_with_missing:
    main_df[col] = main_df[col].fillna(fill_values[col])

In [1171]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 0 to 533
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Position        534 non-null    object 
 1   Name            534 non-null    object 
 2   Salary          534 non-null    int64  
 3   Team            534 non-null    object 
 4   Proj_Runs       534 non-null    float64
 5   OpenOU          534 non-null    float64
 6   Date            534 non-null    object 
 7   Barrel%         534 non-null    float64
 8   Barrels         534 non-null    float64
 9   EV              534 non-null    float64
 10  Events          534 non-null    float64
 11  HR/FB           534 non-null    float64
 12  Hard%           534 non-null    float64
 13  HardHit         534 non-null    float64
 14  HardHit%        534 non-null    float64
 15  ISO             534 non-null    float64
 16  Med%            534 non-null    float64
 17  OPS             534 non-null    flo

In [1172]:
main_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Salary,534.0,2404.681648,560.283860,2000.00,2000.00000,2100.0000,2675.00000,5000.000
Proj_Runs,534.0,4.037154,0.642440,2.48,3.75000,4.0000,4.26000,5.520
OpenOU,534.0,8.051498,0.523973,7.50,7.50000,8.0000,8.50000,9.000
Barrel%,534.0,5.901034,5.539762,0.00,2.30000,4.7960,8.17500,50.000
Barrels,534.0,2.688577,2.880987,0.00,1.00000,1.5500,4.00000,14.000
EV,534.0,87.965251,3.558146,70.30,86.80000,87.2960,90.00000,100.500
Events,534.0,39.147768,25.409539,0.00,26.00000,28.7190,57.75000,102.000
HR/FB,534.0,8.584599,9.641960,0.00,0.00000,6.8180,12.50000,100.000
Hard%,534.0,29.084532,12.243300,0.00,25.00000,27.0200,33.67500,100.000
HardHit,534.0,14.592861,11.388876,0.00,8.00000,9.4820,21.00000,51.000


In [1173]:
#fill in today's date
filepath = r'C:\Users\Owner\FantasySports\MLB_DFS_2022\MLB_Hitting_Live_RFP_FD_2022_08_30.csv'
main_df.to_csv(filepath)

# Next notebook
Go to Hitting_Model_EDA_Modeling